In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [3]:
# Import Libraries
from langchain_openai import ChatOpenAI
from crewai import Agent, Process, Crew, Task
from IPython.display import display, Markdown
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
import os

In [4]:
# Define the LLM
llm = ChatOpenAI(model_name = "gpt-4o", temperature = 0.5)

## User Inputs

In [5]:
# We need to know the Name, company, job position, job description
interviewer = input("Enter the name of the interviewer (if unknown write NONE): ")
company = input("Enter the company: ")
job_position = input("Enter the job position: ")
job_description = input("Enter the job description: ")

## Research Crew

### Research AI Agent

In [6]:
# Research Agent
research_agent = Agent(
    role = 'Research Agent',
    goal = f"Conduct in-depth research to serve as a basis for interview prep",
    backstory = """
    As a Research Specialist, your mission is to uncover detailed and relevant information about the interviewer and company.
        This includes their professional backgrounds, recent projects, social media activity, public speaking engagements, and any insights that can help tailor the interview discussion.
        The ultimate goal is to equip the candidate with backgrounds of the company and the interviewer.
        """,
    llm = llm,
    tools = [ScrapeWebsiteTool(), SerperDevTool()]
)

### Research Tasks

In [7]:
# Research Task
research_company_task = Task(
    description = f"""
    Research about {company}  their position in the sector, and recent industry trends.
    Focus on news and developments from the last year, as the current
        year is 2024.""",
    expected_output=f"A comprehensive report in Markdown with bullet points about {company}",
    agent = research_agent
)

In [9]:
# Research Interviewer Task
research_person_task = Task(
    description = f"Research about {interviewer} with a focus on their professional background, recent projects, social media activity, and any insights that can help tailor the interview discussion.",
    expected_output=f"A comprehensive report in Markdown with bullet points about {interviewer}",
    agent = research_agent)

### AI Coach + Interview Prep

In [10]:
# Research Agent
coach_agent = Agent(
    role = 'coach Agent',
    goal = f"Help prepare the user for job interview at {company} for {job_position}",
    backstory = """
    As an experienced interview coach, your task is to help prepare the user for the interview.
    The user should be asked questions, and be given advice on how to improve the answers.
    """,
    llm = llm,
)

In [11]:
# Prepare list of 8 questions
define_questions_task = Task(
    description = f""" Define a list of questions based on the {company}, {interviewer} and {job_description} for the job {job_position}
    """,
    expected_output=f"A list of 8 questions in Markdown for culture fit and position fit",
    context = [research_company_task, research_person_task],
    agent = coach_agent
)

### Assemble Research Crew

In [ ]:
crew = Crew(
    agents=[research_agent, coach_agent],  # List of agents involved
    tasks=[research_company_task, research_person_task, define_questions_task],  # List of tasks to execute
    verbose=True,
    process = Process.sequential,
)
result = crew.kickoff({"topic": f"draft the question for the interview"})  # Start the task execution

╭─────────────────────────── Crew Execution Started ───────────────────────────╮
│                                                                              │
│  Crew Execution Started                                                      │
│  Name: crew                                                                  │
│  ID: 2484b66c-efcd-4e30-b15d-57d19651cea7                                    │
│                                                                              │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 767286f6-6f0c-48fe-94e4-e5302af805d8
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 767286f6-6f0c-48fe-94e4-e5302af805d8
       Status: Executing Task...
    └── 🤖 Agent: Research Agent
            Status: In Progress

# Agent: Research Agent
## Task: 
    Research about Beat81  their position in the sector, and 

2025-04-21 03:06:06,490 - 6308114432 - __init__.py-__init__:364 - ERROR: Exception while exporting Span batch.
Traceback (most recent call last):
  File "/Users/mdashikadnan/Documents/adnanedu/python/ztm/crewai/crewai_venv/lib/python3.11/site-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "/Users/mdashikadnan/Documents/adnanedu/python/ztm/crewai/crewai_venv/lib/python3.11/site-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/Users/mdashikadnan/Documents/adnanedu/python/ztm/crewai/crewai_venv/lib/python3.11/site-packages/urllib3/connection.py", line 516, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mdashikadnan/Documents/adnanedu/python/ztm/crewai/crewai_venv/lib/python3.11/http/client.py", line 1395, in getresponse
    response.begin()

### Exporting

In [13]:
# Check the crew outcome
display(Markdown(result.raw))

```markdown
# Interview Questions for Data Analytics Working Student at Beat81

## Culture Fit Questions
1. **Can you describe a time when you participated in a team event or challenge? How did it impact your team dynamics and personal growth?**
   - This question assesses the candidate's experience with team culture and their ability to engage in community activities, reflecting Beat81's emphasis on team rituals and community challenges.

2. **How do you prioritize your personal commitments with work commitments, especially in a flexible working environment?**
   - This question evaluates the candidate's ability to manage time effectively, which is crucial given Beat81's offer of flexible working hours.

3. **What attracts you to work in a dynamic and fast-paced start-up environment like Beat81?**
   - This question helps determine the candidate's motivation and readiness to thrive in a start-up culture, which is essential for Beat81's rapidly growing environment.

4. **How do you stay motivated and maintain productivity when working remotely?**
   - This question assesses the candidate's ability to work independently and remain productive, aligning with Beat81's hybrid work model.

## Position Fit Questions
5. **Can you give an example of how you used data analytics to solve a problem or improve a process?**
   - This question evaluates the candidate's practical experience in data analytics, which is critical for supporting Beat81’s daily operations with analytical insights.

6. **Describe your experience with SQL and Python in data analysis projects.**
   - This question assesses the candidate's technical skills in SQL and Python, which are essential for the role.

7. **How do you approach creating and maintaining dashboards for different stakeholders?**
   - This question explores the candidate's ability to communicate data insights effectively through dashboards, a key responsibility in the role.

8. **What experience do you have with data visualization tools like Tableau or Lookerstudio, and how have you used them to convey insights?**
   - This question assesses the candidate's proficiency in data visualization, which is important for presenting data insights to various teams at Beat81.
```

This list of questions is designed to assess both culture fit and position fit for the role of Data Analytics Working Student at Beat81, ensuring that candidates align with the company’s dynamic environment and technical requirements.

In [14]:
# Export each of the tasks output
# Exporting the company report

# Define the file name
file_name = f'{company}_report.txt'

# Open the file in write mode and write the content
with open(file_name, "w", encoding="utf-8") as file:
    file.write(result.tasks_output[0].raw)

print(f"Data successfully exported to {file_name}")


Data successfully exported to Beat81_report.txt


In [15]:
# Export the interviewer report
# Define the file name
file_name = f'{interviewer}_report.txt'

# Open the file in write mode and write the content
with open(file_name, "w", encoding="utf-8") as file:
    file.write(result.tasks_output[1].raw)

print(f"Data successfully exported to {file_name}")

Data successfully exported to Matthias Glöel_report.txt


In [16]:
# Export the interview questions
# Define the file name
file_name = f'{job_position}_interview_questions.txt'

# Open the file in write mode and write the content
with open(file_name, "w", encoding="utf-8") as file:
    file.write(result.tasks_output[2].raw)

print(f"Data successfully exported to {file_name}")


Data successfully exported to Data Analytics Working Student_interview_questions.txt


## Interview Prep

In [17]:
# Import the question from the directory
# Define the file name
file_name = f'{job_position}_interview_questions.txt'

# Open the file in read mode and read the content
with open(file_name, "r", encoding="utf-8") as file:
    questions_data = file.read()

In [18]:
questions_data

"```markdown\n# Interview Questions for Data Analytics Working Student at Beat81\n\n## Culture Fit Questions\n1. **Can you describe a time when you participated in a team event or challenge? How did it impact your team dynamics and personal growth?**\n   - This question assesses the candidate's experience with team culture and their ability to engage in community activities, reflecting Beat81's emphasis on team rituals and community challenges.\n\n2. **How do you prioritize your personal commitments with work commitments, especially in a flexible working environment?**\n   - This question evaluates the candidate's ability to manage time effectively, which is crucial given Beat81's offer of flexible working hours.\n\n3. **What attracts you to work in a dynamic and fast-paced start-up environment like Beat81?**\n   - This question helps determine the candidate's motivation and readiness to thrive in a start-up culture, which is essential for Beat81's rapidly growing environment.\n\n4. **

In [20]:
# AI Agent that asks a question
interviewer_agent = Agent(
    role = 'Interview Agent',
    goal = f"Help prepare the user for job interview at {company} for {job_position}",
    backstory = f"""
    As an experienced interviewer like {interviewer}, your task is to help prepare the user for the interview.
    The user should be asked questions, and be given advice on how to improve the answers.
    """,
    llm = llm,
)

In [21]:
# Interview Task
# Prepare list of 8 questions
asking_task = Task(
    description = f""" ask one random question from {questions_data} from the defined list of questions in Markdown for culture fit and position fit in the knowledge source
    """,
    expected_output=f"A new question to the user",
    agent = interviewer_agent,
    human_input = True
)

In [22]:
# Feedback Task
# Grading Questions
feedback_task = Task(
    description = f""" a list of positives and negatives from the user's answer to the question
    """,
    expected_output=f"feedback of the user's input to the question",
    context = [asking_task],
    agent = coach_agent,
    human_input = True
)

### Interview Prep Crew

In [23]:
# Assemble the interview prep crew
crew = Crew(
    agents=[coach_agent, interviewer_agent],  # List of agents involved
    tasks=[asking_task, feedback_task],  # List of tasks to execute
    verbose=True,
    rocess = Process.hierarchical,
    manager_agent = coach_agent,
    cache = True,
    memory = True,
    planning = True
)
result = crew.kickoff({"topic": "Ask me a question from the knowledge_sources"})  # Start the task execution

╭─────────────────────────── Crew Execution Started ───────────────────────────╮
│                                                                              │
│  Crew Execution Started                                                      │
│  Name: crew                                                                  │
│  ID: 5fb2bd00-c04b-41f7-b1ca-60ebcda5ba88                                    │
│                                                                              │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

 
[2025-04-21 03:06:05][INFO]: Planning the crew execution
🚀 Crew: crew
└── 📋 Task: e9151744-e22c-454c-9c08-9ac5a27e1408
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: e9151744-e22c-454c-9c08-9ac5a27e1408
       Status: Executing Task...
    └── 🤖 Agent: Task Execution Planner
            Status: In Progress

🚀 Crew: crew
└── 📋 Task: e91